# Explore Simple Neural Networks with PyTorch
Copyright 2021, LEAKY.AI LLC

In this exercise, we will build some very simple neural networks using PyTorch and explore their properties.
- Use Google Colab to develop simple neural networks using PyTorch
- Understand how to construct neural networks and estimate their size

To get started, head over to Google Colab via this link:

https://colab.research.google.com

Then load the GitHub project using the following URL:
[FILL OUT] https://github.com/LeakyAI/BirdDetector.git

Good luck!

## Step 1 - Import the PyTorch Libraries
PyTorch has several libraries we will need to build this project.  The main library is the torch library.  We will also load the torchvision library which contains the pre-trained neural network we will need for our project as well as some transformation libraries that will help us process the image before passing it to our neural network.


In [1]:
# Import PyTorch, NN module and NN.functional
import torch
import torch.nn as nn
import torch.nn.functional as F

#random.seed(3) # Python Random package
#torch.manual_seed(34)  # Reproducible results

## Step 2 - Build a Simple Model
Here you will build a simple neural network

In [2]:
# Build a transformation for each image passed into our network
class MySimpleNetwork(nn.Module):
    
        # Network takes 2 inputs, produces 1 output
        def __init__(self):
            super(MySimpleNetwork,self).__init__()
            self.fc1 = nn.Linear(2,1)
            
        def forward(self, x):
            out = self.fc1(x)
            yHat = F.relu(out)
            return yHat
    
# Create an instance of the model and print out summary
net = MySimpleNetwork()

In [3]:
# Inspect the Model
net

MySimpleNetwork(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
)

In [4]:
# Check if model is loaded on GPU or CPU
next(net.parameters()).is_cuda

False

### Weights and Bias
You can read more about weight initialization in PyTorch here:

https://discuss.pytorch.org/t/how-are-layer-weights-and-biases-initialized-by-default/13073/4
and
https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch

In [5]:
print (f"Weights: {net.fc1.weight.data} Bias: {net.fc1.bias.data}")

Weights: tensor([[-0.4584,  0.5515]]) Bias: tensor([-0.6907])


In [6]:
# print out the weights
for param in net.parameters():
  print(param.data)

tensor([[-0.4584,  0.5515]])
tensor([-0.6907])


## Pass a Value into the Network
Since the network has not been trained yet, the output will be garbage.  However, lets see how we would process an input and inspect the output.

In [12]:
# The network takes two inputs
inp = torch.tensor([[-2.0,9.0]])

# Process the input
out = net(inp)
print (out.item())

0.3947283923625946


## Check the calculations

In [8]:
1.0*0.5335 + 2.0*0.2599 -0.4723


0.5810000000000002

# Pause here and continue watching the lesson

## Step 3 - Build a Multi-Layer Model


In [9]:
# Build a transformation for each image passed into our network
class MySimpleNetwork(nn.Module):
    
        # Network takes 2 inputs, produces 1 output
        def __init__(self):
            super().__init__()
            self.fc1 = nn.Linear(2,10)
            self.fc2 = nn.Linear(10,1)
        
        def forward(self, x):
            outFC1 = F.relu(self.fc1(x))
            outFC2 = F.relu(self.fc2(outFC1))
            return outFC2

# Create an instance of the model and print out summary
net = MySimpleNetwork()
net

MySimpleNetwork(
  (fc1): Linear(in_features=2, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)

## Key Takeaways
- 
## Next Steps
- 